##### aimldl >python3 > packages > tensorflow > tutorials > beginners > 1_1_3-save_and_load_models-MNIST.ipynb

* This notebook is a replica of [Save and load models](https://www.tensorflow.org/tutorials/keras/save_and_load) with some comments.
* It is prerequisite to install TensorFlow 2.0.
* See [1_1_1-beginner_quickstart-MNIST.ipynb](#1_1_1-beginner_quickstart-MNIST.ipynb) before this notebook.

Model progress can be saved during—and after—training. This means:
* a model can resume where it left off and avoid long training times.
* you can share your model and others can recreate your work.

When publishing research models and techniques, most machine learning practitioners share:
* code to create the model, and
* the trained weights, or parameters, for the model

### Setup
#### Install HDF5
Models are saved in HDF5 format.

In [1]:
!pip install -q pyyaml h5py 

#### Imports

In [2]:
from __future__ import absolute_import, division, print_function, unicode_literals

import os
import tensorflow as tf
from tensorflow import keras

print(tf.version.VERSION)

2.0.0


### Example: the MNIST dataset
To demonstrate how to save and load weights in HDF5 format, the MNIST dataset is employed.

In [4]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

From TensorFlow Dataset, the MNIST dataset is loaded. In the tutorial, a single line is used as follows:
```python
(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.mnist.load_data()
```
In some examples, the variable names are different, but mean the same things.

| Type of Data   | Notation 1 | Notation 2   |
|----------------|------------|--------------|
| Training Data  | x_train    | train_images |
| Training Label | y_train    | train_labels |
| Test Data      | x_test     | test_images  |
| Test Label     | y_test     | test_labels  |

In this example, only the first 1000 examples are used to speed up the runs.

In [5]:
# Use only a part of the dataset
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

# Normalize the values from [0,255] to [0,1]
train_images = train_images[:1000].reshape(-1, 28 * 28) / 255.0
test_images = test_images[:1000].reshape(-1, 28 * 28) / 255.0

Or simply
```python
train_images = train_images[:1000] / 255.0
test_images = test_images[:1000] / 255.0
```

### Define a model

In [8]:
model = tf.keras.models.Sequential([
  tf.keras.layers.Dense(512, activation='relu', input_shape=(784,)),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

Alternatively, you can define and declare a function. The tutorial has done it this way. We'll define the above lines as a function for later usage.

In [20]:
# Define a simple sequential model
def create_model():
  '''
  # To create a basic model instance, 
  model = create_model()
  '''  
  model = tf.keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(784,)),
    keras.layers.Dropout(0.2),
    keras.layers.Dense(10, activation='softmax')
  ])

  model.compile(optimizer='adam',
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])

  return model

In [9]:
# Display the model's architecture
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 512)               401920    
_________________________________________________________________
dropout (Dropout)            (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                5130      
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


### Save checkpoints during training
The [tf.keras.callbacks.ModelCheckpoint](https://www.tensorflow.org/api_docs/python/tf/keras/callbacks/ModelCheckpoint) callback allows to continually save the model both during and at the end of training.

#### Checkpoint callback usage

In [12]:
checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
print( checkpoint_dir )

training_1


In [13]:
# Create a callback that saves the model's weights
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

In [14]:
cp_callback

In [15]:
# Train the model with the new callback
model.fit(train_images, 
          train_labels,  
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])  # Pass callback to training

# This may generate warnings related to saving the state of the optimizer.
# These warnings (and similar warnings throughout this notebook)
# are in place to discourage outdated usage, and can be ignored.

Train on 1000 samples, validate on 1000 samples
Epoch 1/10
  32/1000 [..............................] - ETA: 0s - loss: 0.0355 - accuracy: 1.0000
Epoch 00001: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 96us/sample - loss: 0.0302 - accuracy: 1.0000 - val_loss: 0.4009 - val_accuracy: 0.8730
Epoch 2/10
 992/1000 [============================>.] - ETA: 0s - loss: 0.0239 - accuracy: 0.9990
Epoch 00002: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 96us/sample - loss: 0.0244 - accuracy: 0.9990 - val_loss: 0.4056 - val_accuracy: 0.8690
Epoch 3/10
  32/1000 [..............................] - ETA: 0s - loss: 0.0354 - accuracy: 1.0000
Epoch 00003: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 89us/sample - loss: 0.0247 - accuracy: 1.0000 - val_loss: 0.4175 - val_accuracy: 0.8670
Epoch 4/10
  32/1000 [..............................] - ETA: 0s - loss: 0.0166 - accuracy: 1.0000
Epoch 00

Notice the message shows "saving model to" which indicates the checkpoint files are updated at the end of each epoch.
```
Epoch 00001: saving model to training_1/cp.ckpt
Epoch 00002: saving model to training_1/cp.ckpt
  ...
Epoch 00010: saving model to training_1/cp.ckpt
```
As a result, the models are saved to sub-directory train_1.

In [17]:
!ls training_1

checkpoint  cp.ckpt.data-00000-of-00001  cp.ckpt.index


In [18]:
!ls -al training_1

total 4792
drwxr-xr-x 2 aimldl aimldl    4096 11월 21 11:36 .
drwxrwxr-x 4 aimldl aimldl    4096 11월 21 11:41 ..
-rw-r--r-- 1 aimldl aimldl      71 11월 21 11:36 checkpoint
-rw-r--r-- 1 aimldl aimldl 4886688 11월 21 11:36 cp.ckpt.data-00000-of-00001
-rw-r--r-- 1 aimldl aimldl    1222 11월 21 11:36 cp.ckpt.index


#### A New Untrained Model
Now a new model is created which is not trained. The weights are at the initial values. The test result with this untrained network is bad. The accuracy is only 11.40%. If another new model is created, the accuracy will be different because the initial weights are random. The common part is the accuracy is very low at the beginning.

In [22]:
# A new basic model instance without training
new_model = create_model()

# Evaluate the model
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 2.2810 - accuracy: 0.1140
Untrained model, accuracy: 11.40%


#### Load the Weights to the New Model from the Checkpoint

In [23]:
new_model.load_weights( checkpoint_path )

In [24]:
# Re-evaluate the model after loading the checkpoint.
loss, acc = new_model.evaluate(test_images,  test_labels, verbose=2)
print("Untrained model, accuracy: {:5.2f}%".format(100*acc))

1000/1 - 0s - loss: 0.4446 - accuracy: 0.8720
Untrained model, accuracy: 87.20%


The accuracy of the loaded model is 87.20%. Recall the accuracy at epoch 10 of the trained model was 100%.
```
Epoch 00010: saving model to training_1/cp.ckpt
1000/1000 [==============================] - 0s 94us/sample - loss: 0.0086 - accuracy: 1.0000 - val_loss: 0.4215 - val_accuracy: 0.8720
```

### Checkpoint callback options
The default tensorflow format only saves the 5 most recent checkpoints. The callback provides several options to adjust the checkpointing frequency & name each checkpoint differently.

#### Assigning Name of Each Checkpoint Differently

In [32]:
# Include the epoch in the file name (uses `str.format`)
checkpoint_path = "training_2/cp-{epoch:04d}.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)
print(f'{checkpoint_path}' )
print( checkpoint_path.format(epoch=0) )
print(f'{checkpoint_dir}')

training_2/cp-{epoch:04d}.ckpt
training_2/cp-0000.ckpt
training_2


Note the sub-directory name has changed to training_2, not training_1.

#### Adjust the Checkpointing Frequency

In [37]:
# Create a callback that saves the model's weights every 5 epochs
cp_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_path, 
    verbose=1, 
    save_weights_only=True,
    period=5)

```python
WARNING:tensorflow:`period` argument is deprecated. Please use `save_freq` to specify the frequency in number of samples seen.
```
Notice save_freq=5 is not the same as period=5.

#### Create a Model Saving Weights Every 5 Epoches
Create a new model which saves uniquely named checkpoints at every five epochs as configured above.

In [38]:
# Create a new model instance
model5 = create_model()

# Save the weights using the `checkpoint_path` format
model5.save_weights(checkpoint_path.format(epoch=0))

##### Train the Model with the New Callback (Options)

In [41]:
# Train the model with the new callback
model.fit( train_images, 
           train_labels,
           epochs=50, 
           callbacks=[cp_callback],
           validation_data=(test_images,test_labels),
           verbose=0 )


Epoch 00005: saving model to training_2/cp-0005.ckpt

Epoch 00010: saving model to training_2/cp-0010.ckpt

Epoch 00015: saving model to training_2/cp-0015.ckpt

Epoch 00020: saving model to training_2/cp-0020.ckpt

Epoch 00025: saving model to training_2/cp-0025.ckpt

Epoch 00030: saving model to training_2/cp-0030.ckpt

Epoch 00035: saving model to training_2/cp-0035.ckpt

Epoch 00040: saving model to training_2/cp-0040.ckpt

Epoch 00045: saving model to training_2/cp-0045.ckpt

Epoch 00050: saving model to training_2/cp-0050.ckpt


The training is checkpointed every 5 epoch. The new checkpoint_dir or training_2 has the following files.

In [42]:
!ls {checkpoint_dir}

checkpoint			  cp-0030.ckpt.data-00000-of-00001
cp-0005.ckpt.data-00000-of-00001  cp-0030.ckpt.index
cp-0005.ckpt.index		  cp-0035.ckpt.data-00000-of-00001
cp-0010.ckpt.data-00000-of-00001  cp-0035.ckpt.index
cp-0010.ckpt.index		  cp-0040.ckpt.data-00000-of-00001
cp-0015.ckpt.data-00000-of-00001  cp-0040.ckpt.index
cp-0015.ckpt.index		  cp-0045.ckpt.data-00000-of-00001
cp-0020.ckpt.data-00000-of-00001  cp-0045.ckpt.index
cp-0020.ckpt.index		  cp-0050.ckpt.data-00000-of-00001
cp-0025.ckpt.data-00000-of-00001  cp-0050.ckpt.index
cp-0025.ckpt.index


The latest checkpoint can be obtained conveniently as:

In [45]:
latest = tf.train.latest_checkpoint(checkpoint_dir)
latest

'training_2/cp-0050.ckpt'

In [46]:
type(latest)

str

(EOF)

TODO: Fix the following error